In [ ]:
from this import s
import pandas as pd
import json

In [ ]:
import os
if 'COLAB_GPU' in os.environ:
    from google.colab import  drive
    drive.mount('/drive')
    data_path = '/drive/Shared drives/Capstone/notebooks/data'
else:
    data_path = 'data'

Drive already mounted at /drive; to attempt to forcibly remount, call drive.mount("/drive", force_remount=True).


In [ ]:
pop_df = pd.read_csv(f'{data_path}/processed/counties_merged_FIPS_str.csv')

pop_df = pop_df[['population','FIPS', 'County_Name']]
pop_df[pop_df['FIPS']==48121]['population'].iloc[0]

833822.0

In [ ]:


# need to get all the crime data into the format
# <state>,<county>,<year1>,<year2>,..
# and later the fips data

# some agencies are split over two counties

# I'm going to focus only on 2019 data. We could expand beyond this, but 
# that's all I've got the energy for now

if __name__ == '__main__':
    states = ['AL','AK','AZ','AR','CA','CO','CT','DE','FL','GA','HI','ID','IL','IN','IA','KS','KY','LA','ME','MD','MA','MI','MN','MS','MO','MT','NE','NV','NH','NJ','NM','NY','NC','ND','OH','OK','OR','PA','RI','SC','SD','TN','TX','UT','VT','VA','WA','WV','WI','WY']

    years = []
    for i in range(1985,2021):
        years.append(i)

    f1 = open(f"{crime_data}/Crime_Data/state_county_fips.json")
    state_county_fips = json.load(f1)

    state_county_crimes = {}

    cols = []
    cols.append('STATE')
    cols.append('COUNTY')
    cols.append('FIPS')
    [cols.append(y) for y in years]

    records = []
    df = pd.DataFrame()

    for state in states:
        f2 = open(f"{data_path}/raw/crime_data/{state}_ori_violentcrimes.json")
        
        df_state = pd.read_json(f"{data_path}/raw/crime_data/{state}_ori_violentcrimes.json").T

        # 

        df = pd.concat([df_state, df])


    

    print('done')

done


In [ ]:
df[['ori', 'agency_name', 'agency_type_name', 'state_name', 'state_abbr',
        'county_name','violent_crime_counts']].sample(20)

# df['state_abbr'].value_counts() # number of agencies in each state

for x in df[df['agency_name']=='Wind River Agency']['violent_crime_counts']:
  print(x)



[[2006, 72], [2007, 88], [2008, 92], [2009, 66], [2010, 51], [2011, 82], [2012, 40], [2013, 43], [2014, 74], [2015, 108], [2016, 152], [2017, 120], [2018, 68], [2019, 78], [2020, 95]]


In [ ]:
df['fips_keys']=''
df['population']=''
df['2019_crimes']=''
df['weighted_vals']=''
df['weighted_crimes_2019']=''


In [ ]:
print(len(df))
print(len(df[df['county_name'].notnull()]))

18616
18616


In [ ]:
# print(len(df))
# df = df[df['county_name'].notnull()]

# for x in df[df['county_name']=='HURON; RICHLAND']['violent_crime_counts']: # this is an agency that reports 2+ counties that no longer reports data
#   print(x)

In [ ]:
count_len_greater_1_no_2019_data = 0  # should be about 52
count_len_greater_1_with_2019_data = 0 #576 
total = 0

for ind, row in df.iterrows():
  counties = (row['county_name'].split(';'))
  for val in row['violent_crime_counts']: # get number of 2019 crimes
    if val[0] ==2019:
      row['2019_crimes'] = (val[1])
  if len(counties) > 1: # only look at agencies that report to more than one county
      values = []
      for county in counties: 
        fips_key = (row['state_abbr']+ '-' + county.strip())
        if fips_key in list(state_county_fips.keys()):
                values.append(state_county_fips[fips_key])
      row['fips_keys'] = values
      
      pop = []

      for fips_key in row['fips_keys']: 
        if len(fips_key) > 1:
          fips_key = int(fips_key)
          pop.append(pop_df[pop_df['FIPS']==fips_key]['population'].iloc[0])
      row['population'] = pop
      weighted_crimes = []

      for num in range(len(row['population'])):
        try:    
          weighted_crimes.append(row['population'][num]*row['2019_crimes']/sum(row['population']))
          count_len_greater_1_with_2019_data += (1/ len(row['population']))
        except:
          weighted_crimes.append(0)
          count_len_greater_1_no_2019_data += (1/ len(row['population'])) # add one to number of agencies with no 2019 data

      row['weighted_crimes_2019']=weighted_crimes
  else:
    for county in counties: 
      fips_key = (df['state_abbr'][ind] + '-' + county.strip())

      try:
        row['fips_keys'] = state_county_fips[fips_key]
      except:
        total +=1
print(total)
    

  
df.head()

455


,ori,agency_name,agency_type_name,state_name,state_abbr,division_name,region_name,region_desc,county_name,nibrs,latitude,longitude,nibrs_start_date,violent_crime_counts,fips_keys,population,2019_crimes,weighted_vals,weighted_crimes_2019
WY0010000,WY0010000,Albany County Sheriff's Office,County,Wyoming,WY,Mountain,West,Region IV,ALBANY,False,41.31085,-105.59061,None,"[[1985, 14], [1986, 20], [1987, 25], [1988, 33...",56001,,6,,
WY0010100,WY0010100,Laramie Police Department,City,Wyoming,WY,Mountain,West,Region IV,ALBANY,False,41.665514,-105.721883,None,"[[1985, 16], [1986, 10], [1987, 32], [1988, 31...",56001,,58,,
WY0010200,WY0010200,University of Wyoming,University or College,Wyoming,WY,Mountain,West,Region IV,ALBANY,False,41.665514,-105.721883,01/01/2021,"[[1985, 3], [1986, 3], [1987, 2], [1988, 4], [...",56001,,3,,
WY0020000,WY0020000,Big Horn County Sheriff's Office,County,Wyoming,WY,Mountain,West,Region IV,BIG HORN,False,44.525654,-107.99332,None,"[[1985, 1], [1986, 2], [1987, 8], [1988, 8], [...",56003,,3,,
WY0020100,WY0020100,Basin Police Department,City,Wyoming,WY,Mountain,West,Region IV,BIG HORN,False,44.525654,-107.99332,None,"[[1985, 1], [1986, 0], [1987, 5], [1988, 7], [...",56003,,,,


In [ ]:
pd.to_numeric(df['2019_crimes']).sum()

1186666.0

In [ ]:
# retrieve the number of violent crimes reported to tribal agencies. 
pd.to_numeric(df[df['agency_type_name']=='Tribal']['2019_crimes']).sum()


7575.0

In [ ]:
df = df[df['agency_type_name']!='Tribal'] # exclude tribal agencies from here on out

pd.to_numeric(df[df['county_name']=='']['2019_crimes']).sum() # number of violent crimes for other excluded data

52772.0

In [ ]:
df[df['county_name']=='']['agency_type_name'].value_counts() # which agency types don't have any data?

State Police             138
Other State Agency        78
University or College     20
Other                      8
City                       2
Name: agency_type_name, dtype: int64

In [ ]:
pd.to_numeric(df[df['county_name']!='']['2019_crimes']).sum()

1126319.0

In [ ]:
import numpy as np
df['county_name']=df['county_name'].str.split(';')
df = df.replace(r'^\s*$', np.nan, regex=True) # make missing values nan
df2 = df.loc[df['population'].notnull()]


In [ ]:
df[df['population'].isnull()] # this is the number of counties with only one county (or no counties) associated with the data



# 'Lytle Police Department', 
# Medina County Sheriff's Office
# ['MEDINA']
# Castroville Police Department
# ['MEDINA']
# Devine Police Department
# ['MEDINA']
# Hondo Police Department
# ['MEDINA']
# Lacoste Police Department
# ['MEDINA']
# Natalia
# ['MEDINA', ' BEXAR']
# San Antonio Police Department







,ori,agency_name,agency_type_name,state_name,state_abbr,division_name,region_name,region_desc,county_name,nibrs,latitude,longitude,nibrs_start_date,violent_crime_counts,fips_keys,population,2019_crimes,weighted_vals,weighted_crimes_2019
TX0070100,TX0070100,Lytle Police Department,City,Texas,TX,West South Central,South,Region III,"[BEXAR, MEDINA, ATASCOSA]",True,28.891397,-98.535381,01/01/2018,"[[1994, 14], [1995, 3], [1996, 1], [1997, 3], ...","[48029, 48325, 48013]","[1952843.0, 50057.0, 49528.0]",1,NaN,"[0.9514794185228422, 0.024389162494372518, 0.0..."


In [ ]:
single_county_df = df.loc[df['population'].isnull()]

single_county_df['final_2019_crimes'] = pd.to_numeric(single_county_df['2019_crimes'])



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
df2.head()
# exploded df contains all of the agencies with multiple counties 
# with weighted crime splits, by population in weighted_crimes_2019
exploded_df = df2.explode(['county_name','fips_keys','weighted_crimes_2019','population'])
exploded_df['final_2019_crimes']=exploded_df['weighted_crimes_2019']
exploded_df.head()

,ori,agency_name,agency_type_name,state_name,state_abbr,division_name,region_name,region_desc,county_name,nibrs,latitude,longitude,nibrs_start_date,violent_crime_counts,fips_keys,population,2019_crimes,weighted_vals,weighted_crimes_2019,final_2019_crimes
WI0050300,WI0050300,Pulaski Police Department,City,Wisconsin,WI,East North Central,Midwest,Region II,SHAWANO,False,44.473961,-87.995926,None,"[[1990, 3], [1991, 3], [1992, 3], [1993, 1], [...",55115,40904.0,8,NaN,1.082575,1.082575
WI0050300,WI0050300,Pulaski Police Department,City,Wisconsin,WI,East North Central,Midwest,Region II,BROWN,False,44.473961,-87.995926,None,"[[1990, 3], [1991, 3], [1992, 3], [1993, 1], [...",55009,261368.0,8,NaN,6.917425,6.917425
WI0090400,WI0090400,Stanley Police Department,City,Wisconsin,WI,East North Central,Midwest,Region II,CLARK,True,45.069092,-91.283505,01/01/2019,"[[1985, 1], [1986, 3], [1987, 2], [1988, 6], [...",55019,34579.0,5,NaN,1.755796,1.755796
WI0090400,WI0090400,Stanley Police Department,City,Wisconsin,WI,East North Central,Midwest,Region II,CHIPPEWA,True,45.069092,-91.283505,01/01/2019,"[[1985, 1], [1986, 3], [1987, 2], [1988, 6], [...",55017,63892.0,5,NaN,3.244204,3.244204
WI0100400,WI0100400,Colby-Abbotsford Police Department,City,Wisconsin,WI,East North Central,Midwest,Region II,MARATHON,False,44.733596,-90.610201,None,"[[1996, 2], [1997, 10], [1999, 2], [2000, 7], ...",55073,135396.0,2,NaN,1.593128,1.593128


In [ ]:
all_agencies_df = pd.concat([exploded_df, single_county_df])

final_df = all_agencies_df.groupby('fips_keys').sum().reset_index()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: Dropping invalid columns in DataFrameGroupBy.add is deprecated. In a future version, a TypeError will be raised. Before calling .add, select only columns which should be valid for the function.
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
final_df = final_df[['fips_keys', 'final_2019_crimes']]
pop_df['FIPS'] = pd.to_numeric(pop_df['FIPS'])
final_df['fips_keys'] = pd.to_numeric(final_df['fips_keys'])


final_df= pd.merge(final_df,pop_df,how='left', left_on='fips_keys',right_on='FIPS')

In [ ]:
final_df['crime_rate']=final_df['final_2019_crimes']/final_df['population']*100000

final_df[final_df['County_Name']=='MEDINA']

,fips_keys,final_2019_crimes,population,FIPS,County_Name,crime_rate
2084,39103,74.605212,177980.0,39103.0,MEDINA,41.91775
2675,48325,431.088907,50057.0,48325.0,MEDINA,861.19605


In [ ]:
final_df = final_df[['fips_keys', 'final_2019_crimes','crime_rate']]

final_df.to_csv(f'{data_path}/processed/weighted_crime_data_fips_split.csv', index=False)